In [0]:
import yaml

with open('../../configs/research_agent.yaml', 'r') as file:
    config = yaml.safe_load(file)

#load global configs
databricks_config = config['databricks_config']

#load uc configs
catalog=databricks_config['catalog']
schema=databricks_config['schema']
eval_table = databricks_config['eval_table_name']

# Example Eval Schema

```
eval_dataset_list = [
    {
        "inputs": {
            "messages": [
                {
                    "role": "user",
                    "content": "What was American Express's total revenue in 2020?",
                }
            ]
        },
        "expectations": {
            "expected_response": "$36.1 billion"
        
        },
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "What was AAPL's operating income in 2022?"}
            ]
        },
        "expectations": {
            "expected_response": "$119,437 million or $119.4 billion"
        
        },
    },
    {
        "inputs": {
            "messages": [
                {
                    "role": "user",
                    "content": "What was the quick ratio for American express in 2022? What factors contributed to this?",
                }
            ]
        },
        "expectations": {
            "expected_facts:": ["7.02", "30% growth in customer deposits", "reduced short term borrowing"]
        },
    },
    {
        "inputs": {
            "messages": [
                {
                    "role": "user",
                    "content": "How did APPL's operating margin change between 2020 and 2021? What factors contributed to this?",
                }
            ]
        },
        "expectations": {
            "expected_facts:": ["3.2 percentage points", "exceptional sales growth in the iphone, mac, and ipad business"]
        },
    },
]
```

In [0]:
import os
evals = spark.read.format("csv").option("header", "true").option("delimiter", "\t").load(f"file://{os.getcwd()}/finance_bench_evals.tsv")
display(evals)

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import split, struct, col

# starting df: inputs, expected_response
evals_final = (
    evals.select(
        # ── build the {"messages": [{"role":"user","content": <inputs>}] } struct ──
        F.struct(                                                            
            F.array(                                                         
                F.struct(                                                   
                    F.lit("user").alias("role"),                            
                    F.col("request").alias("content")                        
                )
            ).alias("messages")                                             
        ).alias("inputs"),                                                  

        # ── wrap expected_response ─────────────────────────────────────────
        F.struct(col("expected_response")
    ).alias("expectations")
    )
)

In [0]:
display(evals_final)

In [0]:
spark.sql(f"DROP TABLE IF EXISTS `{catalog}`.{schema}.{eval_table}")

In [0]:
evals_final.write.mode("overwrite").saveAsTable(f"`{catalog}`.{schema}.{eval_table}")

In [0]:
display(spark.table(f"`{catalog}`.{schema}.{eval_table}"))